Step 1: Import Necessary Libraries

In [60]:
import os
import json
import pandas as pd
import numpy as np
from glob import glob
import re

Step 2: Define a Function to Extract Information from File Names

In [61]:
def extract_info_from_filename(filename):
    pattern = r'(?P<data>CIFAR10|CIFAR100|MNIST|Texture|Defect|Concrete)_(?P<model>ResNet18|ResNet50|VGG16)_(?P<type>MP|RBF|Sausage|ENN|ENN_no_initialization)_maxepochs.*\.json'
    match = re.match(pattern, filename)
    if match:
        return match.group('data'), match.group('model'), match.group('type')
    else:
        return None, None, None

Step 3: Read JSON Files and Extract Data

In [62]:
def read_json_files(file_path_pattern, key = 'test_accuracy'):
    files = glob(file_path_pattern)
    data_list = []

    for file in files:
        with open(file, 'r') as f:
            content = json.load(f)
            data, model, type_ = extract_info_from_filename(os.path.basename(file))
            if data and model and type_:
                newlist = [x*100 for x in content[key] if x != np.nan] # clean list from NaN
                data_list.append({
                    'data': data,
                    'model': model,
                    'type': type_,
                    'values': newlist  # Assuming 'values' is a key in your JSON
                })

    return data_list

Step 4: Compute Average and Standard Error

In [63]:
def compute_statistics(data_list):
    df = pd.DataFrame(data_list)
    df['mean'] = df['values'].apply(np.mean)
    df['stderr'] = df['values'].apply(lambda x: np.std(x, ddof=1) / np.sqrt(len(x)))
    df.drop(columns = 'values', inplace = True)
    return df

Step 5: Organize Data into a Pandas DataFrame

In [64]:
def format_values(df):
    # create mean +- stderr format column
    df['formatted'] = df.apply(lambda row: f"{row['mean']:.5f} ± {row['stderr']:.8f}", axis=1)
    df.drop(columns = ['mean', 'stderr'], inplace = True)
    return df

def create_final_dataframe(df):
    formatted_df = format_values(df)
    formatted_df.set_index(['type','model', 'data'], inplace=True)
    return formatted_df.unstack(-1)

Putting It All Together, first for 20 epochs results

In [65]:
# Define the pattern for your JSON files
file_path_pattern = 'W:/DS/Project/CNN Experiment/20_epochs_results/*.json'

# Create the DataFrame
data_list = read_json_files(file_path_pattern)
df = compute_statistics(data_list)
final_df = create_final_dataframe(df)

In [74]:
# Display the final DataFrame
print(final_df['formatted']['CIFAR100'])

type                   model   
ENN                    ResNet18    64.17935 ± 0.09762427
                       ResNet50    70.87440 ± 0.03210827
                       VGG16       56.35350 ± 0.17966931
ENN_no_initialization  ResNet18    61.67067 ± 0.04784468
                       ResNet50    69.91520 ± 0.09706327
                       VGG16       57.66059 ± 0.13288822
MP                     ResNet18    68.45853 ± 0.04024297
                       ResNet50    74.92822 ± 0.04736327
                       VGG16       62.65525 ± 0.08521473
RBF                    ResNet18    25.98858 ± 0.23883575
                       ResNet50    25.75287 ± 0.34619737
                       VGG16        1.00160 ± 0.00000000
Sausage                ResNet18    44.67899 ± 0.52626853
Name: CIFAR100, dtype: object


Then, for 200 epochs results

In [67]:
# Define the pattern for your JSON files
file_path_pattern = 'W:/DS/Project/CNN Experiment/200_epochs_results/*.json'

# Create the DataFrame
data_list = read_json_files(file_path_pattern)
df_200 = compute_statistics(data_list)
final_df_200 = create_final_dataframe(df_200)

In [68]:
# Display the final DataFrame
print(final_df_200)

                                            formatted
data                                          CIFAR10
type                  model                          
ENN                   ResNet18  87.90899 ± 0.06570042
ENN_no_initialization ResNet18  87.81801 ± 0.04315299
MP                    ResNet18  88.58173 ± 0.09962500
RBF                   ResNet18  88.01082 ± 0.03711763
Sausage               ResNet18  87.51002 ± 0.03517446
